In [1]:
import pandas as pd

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [2]:
df = df.iloc[:300]

In [3]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


## Q1. Getting the embeddings model

Now, get the embeddings model `multi-qa-mpnet-base-dot-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

> Note: this is not the same model as in HW3

```bash
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embeddings for the first LLM answer:

```python
answer_llm = df.iloc[0].answer_llm
```

What's the first value of the resulting vector?

* **-0.42**
* -0.22
* -0.02
* 0.21

In [4]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

answer_llm = df.iloc[0].answer_llm

embedding_answer_llm = embedding_model.encode(answer_llm)
embedding_answer_llm[0]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





-0.42244655

## Q2. Computing the dot product


Now for each answer pair, let's create embeddings and compute dot product between them

We will put the results (scores) into the `evaluations` list

What's the 75% percentile of the score?

* 21.67
* **31.67**
* 41.67
* 51.67

In [5]:
def cosine_similarity(sentence1, sentence2):    
    v_llm = embedding_model.encode(sentence1)
    v_orig = embedding_model.encode(sentence2)
    return v_llm.dot(v_orig) 

def describe(l: list) -> pd.Series:
    l_copy = l.copy()
    ser = pd.Series(l)
    
    return ser

In [6]:
evaluations = []

for i in range (df.shape[0]):        
    evaluations.append(cosine_similarity(df.iloc[i].answer_llm, df.iloc[i].answer_orig))


In [7]:
describe(evaluations).describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
dtype: float64

## Q3. Computing the cosine

From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

So we need to normalize them.

To do it, we 

* Compute the norm of a vector
* Divide each element by this norm

So, for vector `v`, it'll be `v / ||v||`

In numpy, this is how you do it:

```python
norm = np.sqrt((v * v).sum())
v_norm = v / norm
```

Let's put it into a function and then compute dot product 
between normalized vectors. This will give us cosine similarity

What's the 75% cosine in the scores?

* 0.63
* 0.73
* **0.83**
* 0.93

In [8]:
import numpy as np
def norm(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

def cosine_similarity(sentence1, sentence2):    
    v_llm = embedding_model.encode(sentence1)
    v_llm = norm(v_llm)
    v_orig = embedding_model.encode(sentence2)
    v_orig = norm(v_orig)
    
    return v_llm.dot(v_orig) 


In [9]:
evaluations_norm = []

for i in range (df.shape[0]):        
    evaluations_norm.append(cosine_similarity(df.iloc[i].answer_llm, df.iloc[i].answer_orig))


describe(evaluations_norm).describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
dtype: float64

## Q4. Rouge

Now we will explore an alternative metric - the ROUGE score.  

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

```bash
pip install rouge
```

(The latest version at the moment of writing is `1.0.1`)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (`doc_id=5170565b`)

```
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
```

There are three scores: `rouge-1`, `rouge-2` and `rouge-l`, and precision, recall and F1 score for each.

* `rouge-1` - the overlap of unigrams,
* `rouge-2` - bigrams,
* `rouge-l` - the longest common subsequence

What's the F score for `rouge-1`?

- 0.35
- **0.45**
- 0.55
- 0.65

In [10]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [21]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df[df.document=='5170565b']['answer_llm'], df[df.document=='5170565b']['answer_orig'])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [22]:
scores = rouge_scorer.get_scores(df[df.document=='5170565b']['answer_llm'], df[df.document=='5170565b']['answer_orig'])
scores


[{'rouge-1': {'r': 0.45454545454545453,
   'p': 0.45454545454545453,
   'f': 0.45454544954545456},
  'rouge-2': {'r': 0.21621621621621623,
   'p': 0.21621621621621623,
   'f': 0.21621621121621637},
  'rouge-l': {'r': 0.3939393939393939,
   'p': 0.3939393939393939,
   'f': 0.393939388939394}},
 {'rouge-1': {'r': 0.6060606060606061,
   'p': 0.6060606060606061,
   'f': 0.6060606010606061},
  'rouge-2': {'r': 0.43243243243243246,
   'p': 0.41025641025641024,
   'f': 0.42105262658241},
  'rouge-l': {'r': 0.5757575757575758,
   'p': 0.5757575757575758,
   'f': 0.5757575707575758}},
 {'rouge-1': {'r': 0.7878787878787878,
   'p': 0.7222222222222222,
   'f': 0.753623183415249},
  'rouge-2': {'r': 0.6486486486486487,
   'p': 0.5714285714285714,
   'f': 0.6075949317288897},
  'rouge-l': {'r': 0.7575757575757576,
   'p': 0.6944444444444444,
   'f': 0.7246376761688721}},
 {'rouge-1': {'r': 0.8181818181818182,
   'p': 0.6585365853658537,
   'f': 0.7297297247881666},
  'rouge-2': {'r': 0.648648648648

## Q5. Average rouge score

Let's compute the average between `rouge-1`, `rouge-2` and `rouge-l` for the same record from Q4

- 0.35
- 0.45
- 0.55
- 0.65

In [12]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

In [13]:
rouge_avg

0.35490034990035496

## Q6. Average rouge score for all the data points

Now let's compute the score for all the records

```python
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
```

And create a dataframe from them

What's the agerage `rouge_2` across all the records?

- 0.10
- **0.20**
- 0.30
- 0.40

In [23]:
scores_q6 = rouge_scorer.get_scores(df['answer_llm'], df['answer_orig'])


In [24]:
rouge_avg = 0
for c in (scores_q6):
    rouge_avg += c['rouge-2']['f']

rouge_avg / 300

0.20696501983423318